In [1]:
import pandas as pd
import zipfile

In [2]:
# Read Data
zp = zipfile.ZipFile('sessions.csv.zip')
session = pd.read_csv(zp.open('sessions.csv'), index_col = False)
# Subset 3000 rows
session = session.head(3000)
session[['action', 'action_type', 'action_detail']] = session[['action', 'action_type', 'action_detail']].fillna('NaN')
session['secs_elapsed'] = session['secs_elapsed'].fillna(0)
session.head(5)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [16]:
def normalize_cols(df, cols):
    df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0)
    df[cols] = df[cols].fillna(0)


Restart here

In [3]:
action_df = session[['action', 'action_type', 'action_detail', 'user_id']]

g = action_df.groupby(['action', 'action_type', 'action_detail'], as_index=False).count()
g = g[['action', 'action_type', 'action_detail', 'user_id']]
g.columns = ['action', 'action_type', 'action_detail', 'action_count']
# g.head(5)
action_df = pd.merge(action_df, g, how='left', on=['action', 'action_type', 'action_detail'])
action_df.head(5)

,action,action_type,action_detail,user_id,action_count
0,lookup,NaN,NaN,d1mm9tcy42,165
1,search_results,click,view_search_results,d1mm9tcy42,257
2,lookup,NaN,NaN,d1mm9tcy42,165
3,search_results,click,view_search_results,d1mm9tcy42,257
4,lookup,NaN,NaN,d1mm9tcy42,165


In [4]:


action_df['action_gr'] = action_df['action'] + action_df['action_type'] + action_df['action_detail']
action_df.drop(['action', 'action_type', 'action_detail'], axis=1, inplace=True)
action_df.loc[(action_df['action_count'] < 30), 'action_gr'] = 'binned'

In [8]:
action_list = action_df['action_gr'].drop_duplicates().values
action_df['action_gr'] = action_df['action_gr'].replace(" ","_").replace("-","").replace("/","").replace("(","").replace(")","")

In [10]:
print('Starting to split the action feature')

# Pivot WIder on action
for action in action_list:
    action_df[action] = 0
    action_df.loc[(action_df['action_gr'] == action), action] = action_df['action_count']

Starting to split the action feature


In [12]:
action_df.drop(['action_gr', 'action_count'], axis=1, inplace=True)
# For each ID, sum how many actions he has done
action_df = action_df.groupby(['user_id'], as_index=False).sum()

In [15]:
action_df.head(5)

,user_id,lookupNaNNaN,search_resultsclickview_search_results,personalizedatawishlist_content_update,indexviewview_search_results,similar_listingsdatasimilar_listings,ajax_refresh_subtotalclickchange_trip_characteristics,showNaNNaN,showviewp3,binned,...,NaNmessage_postmessage_post,active-unknown--unknown-,update-unknown--unknown-,listings-unknown--unknown-,index-unknown--unknown-,ajax_lwlb_contactclickcontact_host,ajax_check_datesclickchange_contact_host_dates,social_connections-unknown--unknown-,relationship-unknown--unknown-,show-unknown--unknown-
0,11581i5wng,0,514,1316,192,384,0,834,861,43,...,0,0,45,0,0,0,0,0,0,0
1,1ltangt0hg,0,0,4935,0,0,5152,3892,4305,252,...,0,0,585,0,0,0,0,0,0,0
2,4grx6yxeby,0,0,329,0,0,0,0,0,102,...,45,40,0,0,0,0,0,0,0,0
3,4rvqpxoh3h,0,0,0,0,0,0,0,0,14,...,0,40,0,45,240,0,0,0,0,0
4,5f45ro5uzk,660,1542,2303,96,768,1932,1668,2296,45,...,0,0,45,0,0,0,0,0,0,0


In [18]:
# percetage toward the column sum
normalize_cols(action_df, action_list)

Device

In [38]:
device_df = session[['user_id', 'device_type', 'secs_elapsed']]
device_list = device_df['device_type'].drop_duplicates().values

In [40]:
# Pivot wider on device
for device in device_list:
    device_df[device] = 0
    device_df.loc[(device_df['device_type'] == device), device] = device_df['secs_elapsed']

In [41]:
device_df.drop(['device_type', 'secs_elapsed'], axis=1, inplace=True)
device_df = device_df.groupby(['user_id'], as_index=False).sum()
normalize_cols(device_df, device_list)

In [42]:
device_df.head(5)

,user_id,Windows Desktop,-unknown-,Mac Desktop,Android Phone,iPhone,iPad Tablet,Android App Unknown Phone/Tablet
0,11581i5wng,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
1,1ltangt0hg,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
2,4grx6yxeby,0.932994,0.067006,0.0,0.0,0.0,0.0,0.0
3,4rvqpxoh3h,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0
4,5f45ro5uzk,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [43]:
session_df = pd.merge(action_df, device_df, how='outer', on='user_id')

In [44]:
session_df.head(5)

,user_id,lookupNaNNaN,search_resultsclickview_search_results,personalizedatawishlist_content_update,indexviewview_search_results,similar_listingsdatasimilar_listings,ajax_refresh_subtotalclickchange_trip_characteristics,showNaNNaN,showviewp3,binned,...,social_connections-unknown--unknown-,relationship-unknown--unknown-,show-unknown--unknown-,Windows Desktop,-unknown-,Mac Desktop,Android Phone,iPhone,iPad Tablet,Android App Unknown Phone/Tablet
0,11581i5wng,0.000000,0.122702,0.314156,0.045834,0.091669,0.000000,0.199093,0.205538,0.010265,...,0.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
1,1ltangt0hg,0.000000,0.000000,0.258093,0.000000,0.000000,0.269442,0.203546,0.225145,0.013179,...,0.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
2,4grx6yxeby,0.000000,0.000000,0.637597,0.000000,0.000000,0.000000,0.000000,0.000000,0.197674,...,0.0,0.0,0.0,0.932994,0.067006,0.0,0.0,0.0,0.0,0.0
3,4rvqpxoh3h,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041298,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0
4,5f45ro5uzk,0.058124,0.135799,0.202818,0.008454,0.067635,0.170145,0.146896,0.202202,0.003963,...,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0
